In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
from sklearn.svm import LinearSVC
from skimage import feature, exposure

images = []
labels = []
dataset = 'flowers'
#dataset = 'person_car_cup'

# get all the image folder paths
image_paths = os.listdir(f'/content/drive/MyDrive/Colab Notebooks/train/{dataset}')
for path in image_paths:

  # get all the image name
  all_images = os.listdir(f'/content/drive/MyDrive/Colab Notebooks/train/{dataset}/{path}')

  # iterate over the image names, get the label
  for image in all_images:
    image_path = f'/content/drive/MyDrive/Colab Notebooks/train/{dataset}/{path}/{image}'
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 256))

    # get the HOG descriptor for the image
    hog_desc = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
      cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
    
    # update the data and labels
    images.append(hog_desc)
    labels.append(path)

# train Linear SVC 
print('Training on train images...')
svm_model = LinearSVC(random_state=42, tol=1e-5)
svm_model.fit(images, labels)

# predict on the test images
print('Evaluating on test images...')

# loop over the test dataset
for (i, imagePath) in enumerate(os.listdir(f'/content/drive/MyDrive/Colab Notebooks/test/{dataset}/')):
  image = cv2.imread(f'/content/drive/MyDrive/Colab Notebooks/test/{dataset}/{imagePath}')
  resized_image = cv2.resize(image, (128, 256))

  # get the HOG descriptor for the test image
  (hog_desc, hog_image) = feature.hog(resized_image, orientations=9, pixels_per_cell=(8, 8),
    cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys', visualize=True)
  
  # prediction
  pred = svm_model.predict(hog_desc.reshape(1, -1))[0]

  # visualize the HOG image
  hog_image = exposure.rescale_intensity(hog_image, out_range=(0, 255)).astype('uint8')
  cv2_imshow(hog_image)

  # put the predicted text on the test image
  cv2.putText(image, pred.title(), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
  cv2_imshow(image)